In [1]:
# Random
# Popular
# UCB
# TS
# e-Greedy
# Linear UCB
# GLM-UCB
# NICF
# PTS
# ICTRTS
# Cluster Bandit
# WSCB

In [16]:
import numpy as np

In [2]:
from irec.environment.loader.full_data import FullData

In [3]:
from irec.recommendation.agents.simple_agent import SimpleAgent
from irec.recommendation.agents.action_selection_policies.egreedy import ASPEGreedy
from irec.recommendation.agents.action_selection_policies.greedy import ASPGreedy

2022-08-02 15:53:03.888075: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-02 15:53:03.888109: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
from irec.recommendation.agents.value_functions import *

In [22]:
from irec.offline_experiments.evaluation_policies.fixed_interaction import FixedInteraction
from irec.offline_experiments.metric_evaluators.user_cumulative_interaction import UserCumulativeInteraction
from irec.offline_experiments.metric_evaluators.stage_iterations import StageIterations
from irec.offline_experiments.metrics.hits import Hits

In [6]:
# Dataset
dataset = {
    'path': "../dataset/MovieLens 100k/ratings.csv",
    'random_seed': 0,
    'file_delimiter': ",",
    'skip_head': True
}
# Splitting
splitting = {'strategy': "temporal", 'train_size': 0.8, 'test_consumes': 5}
# Loader
loader = FullData(dataset, splitting)
train_dataset, test_dataset, _, _ = loader.process()


Applying splitting strategy: temporal

Test shape: (16892, 4)
Train shape: (83108, 4)


In [7]:
parameters = {
    "Random": {},
    "MostPopular": {},
    "UCB": {"c": 0.25},
    "ThompsonSampling": {"alpha_0": 1, "beta_0": 100},
    "EGreedy": {"epsilon": 0.1},
    "LinearUCB": {"alpha": 1.0, "item_var": 0.01, "iterations": 20, "num_lat": 20, "stop_criteria": 0.0009, "user_var": 0.01, "var": 0.05},
    "GLM_UCB": {"c": 4, "item_var": 0.01, "iterations": 20, "num_lat": 10, "stop_criteria": 0.0009, "user_var": 0.01, "var": 0.05},
#     "NICF": {"batch": 256, "clip_param": 0.2, "dropout_rate": 0.01, "gamma": 0.0, "inner_epoch": 200, "latent_factor": 10, "learning_rate": 0.001, "num_blocks": 1, "num_heads": 2, "restore_model": False, "rnn_layer": 1, "time_step": 100, "training_epoch": 10000},
    "PTS": {"num_lat": 20, "num_particles": 5, "var": 0.5, "var_u": 1.0, "var_v": 1.0},
    "ICTRTS": {"num_lat": 2, "num_particles": 5},
    "ClusterBandit": {"B": 5, "C": 0.5, "D": 3, "num_clusters": 4, "num_lat": 10},
    "WSCB": {"alpha": 0.75, "num_lat": 10}
}

In [8]:
def create_agent(agent_name:str, prmt:dict) -> SimpleAgent:
    value_function = eval(agent_name)(**prmt)
    action_selection_policy = ASPEGreedy(**prmt) if agent_name == "EGreedy" else ASPGreedy()
    agent = SimpleAgent(value_function, action_selection_policy, name=agent_name)
    return agent

In [9]:
agents = {}
for agent_name, prmt in parameters.items():
    agents[agent_name] = create_agent(agent_name, prmt)

In [10]:
eval_policy = FixedInteraction(num_interactions=100, interaction_size=1, save_info=False)

In [ ]:
eval_policy = FixedInteraction(num_interactions=100, interaction_size=1, save_info=False)

In [11]:
interactions = {}
for agent_name, agent in agents.items():
    print(agent_name)
    agent_interactions, action_info = eval_policy.evaluate(agent, train_dataset, test_dataset)
    interactions[agent_name] = agent_interactions

Random
Starting Random Training
Ended Random Training


Random: 100%|████████████████████| 18900/18900 [00:02<00:00, 8033.12it/s]                                                                                                


MostPopular
Starting MostPopular Training
Ended MostPopular Training


MostPopular: 100%|████████████████████| 18900/18900 [00:02<00:00, 8120.59it/s]                                                                                           


UCB
Starting UCB Training
Ended UCB Training


...

ICTRTS: 100%|████████████████████| 18900/18900 [03:35<00:00, 87.70it/s]                                                                                                  


ClusterBandit
Starting ClusterBandit Training
Ended ClusterBandit Training


ClusterBandit: 100%|████████████████████| 18900/18900 [01:10<00:00, 266.77it/s]                                                                                          


WSCB
Starting WSCB Training
0.0 1.0
Ended WSCB Training


WSCB: 100%|████████████████████| 18900/18900 [00:05<00:00, 3471.95it/s]                                                                                                  


In [23]:
# Cumulative Evaluation Setup
evaluator = UserCumulativeInteraction(
    ground_truth_dataset=test_dataset,
    num_interactions=100,
    interaction_size=1,
    interactions_to_evaluate=[5, 10, 20, 50, 100],
    relevance_evaluator_threshold=3.99
)

In [25]:
# Getting the results
cumulative_results = {}
for agent_name, agent_results in interactions.items():
    print(f"\nEvaluating {agent_name}\n")
    hits_values = evaluator.evaluate(metric_class=Hits, results=agent_results)
    cumulative_results[agent_name] = hits_values


Evaluating Random

Computing iteration 5 with StageIterations
Computing iteration 10 with StageIterations
Computing iteration 20 with StageIterations
Computing iteration 50 with StageIterations
Computing iteration 100 with StageIterations
StageIterations spent 0.32 seconds executing Hits metric

Evaluating MostPopular

Computing iteration 5 with StageIterations
Computing iteration 10 with StageIterations
Computing iteration 20 with StageIterations
Computing iteration 50 with StageIterations
Computing iteration 100 with StageIterations
StageIterations spent 0.31 seconds executing Hits metric

Evaluating UCB

Computing iteration 5 with StageIterations
Computing iteration 10 with StageIterations
Computing iteration 20 with StageIterations
Computing iteration 50 with StageIterations
Computing iteration 100 with StageIterations
StageIterations spent 0.37 seconds executing Hits metric

Evaluating ThompsonSampling

Computing iteration 5 with StageIterations
Computing iteration 10 with StageI

In [24]:
# Stage Evaluation Setup
evaluator = StageIterations(
    ground_truth_dataset=test_dataset,
    num_interactions=100,
    interaction_size=1,
    interactions_to_evaluate=[5, 10, 20, 50, 100],
    relevance_evaluator_threshold=3.99
)

In [ ]:
# Getting the results
stage_results = {}
for agent_name, agent_results in interactions.items():
    print(f"\nEvaluating {agent_name}\n")
    hits_values = evaluator.evaluate(metric_class=Hits, results=agent_results)
    results[agent_name] = hits_values